<a href="https://colab.research.google.com/github/4GeeksAcademy/Proyecto_Final_Close-Loop-Intelligence/blob/main/Proyecto_Final_Close_Loop_Intelligence_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Idea Integral Ganadora: Close Loop Intelligence
---

1. ***Definición del Problema (El "Pitch" de Negocio)***

    Muchas empresas sufren de dos males: el stockout (perder ventas por no tener producto) o el exceso de inventario (dinero muerto en estanterías que se deprecia). Este proyecto construye un Sistema de Inteligencia de Ciclo Cerrado que predice cuánto se va a vender para decidir exactamente cuánto comprar y qué promocionar para salir del inventario lento.

2. **Arquitectura de Datos (Paso 2 y 3: Ingesta y SQL)**

    Para llegar a las 60,000+ filas, utilizaremos un dataset transaccional histórico.

    Origen: Dataset de Retail (tipo Online Retail II o Instacart).

    Modelo de Datos (SQL): Almacenarás los datos en una base de datos relacional con este esquema:

        Tabla_Ventas: ID_Transaccion, Fecha, ID_Producto, Cantidad, Precio_Unitario, ID_Cliente.

        Tabla_Productos: ID_Producto, Categoria, Costo_Reposicion, Stock_Actual, Punto_Reorden.

    Uso de SQL (Crítico): No solo cargarás los datos. Usarás SQL para crear una Vista Maestra mediante un JOIN. Calcularás el "Lead Time" (tiempo de entrega) y la "Venta Promedio Diaria" usando GROUP BY y funciones agregadas. Esta vista será la que alimentará tu modelo de Machine Learning.

3. **El Cerebro del Proyecto (Paso 4, 5 y 6: EDA y ML)**

    Aquí es donde ocurre la magia. Dividiremos el modelo en dos enfoques:

    *Enfoque de Ventas (Predicción de Demanda):*

    Utilizarás un modelo de Series Temporales o Regresión (como Random Forest Regressor) para predecir cuántas unidades de un producto se venderán en los próximos 15-30 días basándote en la estacionalidad y tendencias históricas.

    *Enfoque de Compras/Inventario (Optimización):*

    Clasificación ABC: Usarás los datos para clasificar productos. Los "A" son los que más dinero generan (nunca pueden faltar), los "C" son los lentos.

    Cálculo de Stock de Seguridad: Aplicarás fórmulas estadísticas para determinar el inventario mínimo necesario.

4. **La Salida en Ventas: "Revenue Optimizer"**

    Para darle ese toque final de ventas que mencionaste, añadiremos un componente de Estrategia Comercial:

    Si el modelo predice una demanda baja para un producto con mucho stock (Exceso), la aplicación sugerirá automáticamente un descuento o promoción de "cross-selling" para forzar la salida de esa mercancía y recuperar flujo de caja.

5. **Desarrollo de la Web App (Paso 7: La interfaz)**
    
    La aplicación (en Streamlit) tendrá tres pestañas claras para el usuario final:

    *Dashboard Ejecutivo (Ventas):* Gráficos elocuentes que muestran la proyección de ingresos para el próximo mes y el "Top 10" de productos estrella.

    *Módulo de Compras (Reposición):* Una tabla generada por tu modelo que diga: "Debes pedir 500 unidades del Producto X hoy mismo al proveedor Y para no quedar desabastecido el lunes".

    *Módulo de Ofertas (Salida de Inventario):* Una sección que resalte: "¡Alerta! Tienes demasiado stock de este producto. Aplica un 15% de descuento para incentivar la venta".

In [3]:
#Realizamos la instalacion para ejecutar la conexion con Kaggle
!pip install kaggle
!pip install opendatasets

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import opendatasets as od
import sqlite3
import os
import glob

od.download('https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce')

Skipping, found downloaded files in "./brazilian-ecommerce" (use force=True to force download)


In [ ]:
prueba wiston

In [6]:
#Configuramos Nombres de Base de Datos
BD = 'BaseSQL.db'
csv_files = glob.glob('brazilian-ecommerce/*.csv')

conexion = sqlite3.connect(BD)

try:
    print(f"Iniciando migración de {len(csv_files)} archivos a la base de datos...")

    for archivo in csv_files:
        # Limpiamos el nombre para la tabla (sin extensión y sin caracteres raros)
        nombre_tabla = os.path.splitext(archivo)[0].replace(" ", "_").lower()

        # Cargamos el archivo a un DataFrame
        df = pd.read_csv(archivo)

        # 3. Guardar en la base de datos
        # index=False evita que se cree una columna extra para el índice de pandas
        df.to_sql(nombre_tabla, conexion, if_exists='replace', index=False)

        print(f"Cargado con éxito: {archivo} -> Tabla: {nombre_tabla}")

    print("\nProceso finalizado. Ahora puedes borrar los .csv originales si lo deseas.")

finally:
    conexion.close()

Iniciando migración de 9 archivos a la base de datos...
Cargado con éxito: brazilian-ecommerce/olist_order_reviews_dataset.csv -> Tabla: brazilian-ecommerce/olist_order_reviews_dataset
Cargado con éxito: brazilian-ecommerce/olist_products_dataset.csv -> Tabla: brazilian-ecommerce/olist_products_dataset
Cargado con éxito: brazilian-ecommerce/olist_orders_dataset.csv -> Tabla: brazilian-ecommerce/olist_orders_dataset
Cargado con éxito: brazilian-ecommerce/olist_customers_dataset.csv -> Tabla: brazilian-ecommerce/olist_customers_dataset
Cargado con éxito: brazilian-ecommerce/olist_sellers_dataset.csv -> Tabla: brazilian-ecommerce/olist_sellers_dataset
Cargado con éxito: brazilian-ecommerce/olist_order_items_dataset.csv -> Tabla: brazilian-ecommerce/olist_order_items_dataset
Cargado con éxito: brazilian-ecommerce/olist_geolocation_dataset.csv -> Tabla: brazilian-ecommerce/olist_geolocation_dataset
Cargado con éxito: brazilian-ecommerce/product_category_name_translation.csv -> Tabla: brazil

In [10]:
#Consulta SQL
conexion = sqlite3.connect(BD)
cursor = conexion.cursor()

cursor.execute('SELECT name FROM sqlite_master WHERE type = "table"')
tablas = cursor.fetchall()

for tabla in tablas:
    print(tabla[0])

conexion.close()

brazilian-ecommerce/olist_order_reviews_dataset
brazilian-ecommerce/olist_products_dataset
brazilian-ecommerce/olist_orders_dataset
brazilian-ecommerce/olist_customers_dataset
brazilian-ecommerce/olist_sellers_dataset
brazilian-ecommerce/olist_order_items_dataset
brazilian-ecommerce/olist_geolocation_dataset
brazilian-ecommerce/product_category_name_translation
brazilian-ecommerce/olist_order_payments_dataset


In [22]:
# Consulta simple para listar todas las tablas
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Ejecución
df_tablas = pd.read_sql_query(query, conexion)
print(df_tablas)

                                                name
0    brazilian-ecommerce/olist_order_reviews_dataset
1         brazilian-ecommerce/olist_products_dataset
2           brazilian-ecommerce/olist_orders_dataset
3        brazilian-ecommerce/olist_customers_dataset
4          brazilian-ecommerce/olist_sellers_dataset
5      brazilian-ecommerce/olist_order_items_dataset
6      brazilian-ecommerce/olist_geolocation_dataset
7  brazilian-ecommerce/product_category_name_tran...
8   brazilian-ecommerce/olist_order_payments_dataset


In [32]:
#Ejecutamos una consulta simple - DF Order Review

conn = sqlite3.connect(BD)
tables = ['order_reviews', 'products','orders','customers','sellers','order_items','geolocation','order_payments']
dfT = {}

for table in tables:
    query = f'SELECT * FROM "brazilian-ecommerce/olist_{table}_dataset" '
    dfT[table] = pd.read_sql_query(query, conn)
    print(f"Tabla '{table}' cargada con {len(dfT[table])} filas.")

conn.close()


Tabla 'order_reviews' cargada con 99224 filas.
Tabla 'products' cargada con 32951 filas.
Tabla 'orders' cargada con 99441 filas.
Tabla 'customers' cargada con 99441 filas.
Tabla 'sellers' cargada con 3095 filas.
Tabla 'order_items' cargada con 112650 filas.
Tabla 'geolocation' cargada con 1000163 filas.
Tabla 'order_payments' cargada con 103886 filas.


In [34]:
for t in tables:
  print(dfT[t].shape)

(99224, 7)
(32951, 9)
(99441, 8)
(99441, 5)
(3095, 4)
(112650, 7)
(1000163, 5)
(103886, 5)


In [36]:
dfT['customers'].head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [37]:
dfT['order_items'].head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [38]:
dfT['order_payments'].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [39]:
dfT['orders'].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [40]:
dfT['products'].head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [42]:
dfT['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [43]:
dfT['geolocation'].head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [44]:
dfT['order_reviews'].head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
